In [1]:
from joints import joints_dict
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.dom import minidom

In [2]:
def mm2m(dct):
    for i in dct:
        xyz = dct[i]['xyz']
        dct[i]['xyz'] = [round(_ / 1000.0, 6) for _ in xyz]
    return dct

mm2m(joints_dict)  # convert milimeter to meter
link_dict = {}
repo = 'rockerbogie/For_URDF/bin_stl/'        
file_name = 'test.urdf'

In [3]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [4]:
class Link:
    def __init__(self, name, xyz, repo):
        self.name = name
        self.xyz = xyz
        self.xml = None
        self.repo = repo
        
    def gen_link_xml(self):
        """generate the xml strings of links
        """
        self.xyz = [-_ for _ in self.xyz]  # reverse the sign of xyz
        
        link = Element('link')
        link.attrib = {'name':self.name}
        
        # visual
        visual = SubElement(link, 'visual')
        origin_v = SubElement(visual, 'origin')
        origin_v.attrib = {'xyz':' '.join([str(_) for _ in self.xyz]), 'rpy':'0 0 0'}
        geometry_v = SubElement(visual, 'geometry')
        mesh_v = SubElement(geometry_v, 'mesh')
        mesh_v.attrib = {'filename':'package://' + self.repo + self.name + '_m-binary.stl'}
        material = SubElement(visual, 'material')
        material.attrib = {'name':'silver'}
        color = SubElement(material, 'color')
        color.attrib = {'rgba':'1 0 0 1'}
        
        # collision
        collision = SubElement(link, 'collision')
        origin_c = SubElement(collision, 'origin')
        origin_c.attrib = {'xyz':' '.join([str(_) for _ in self.xyz]), 'rpy':'0 0 0'}
        geometry_c = SubElement(collision, 'geometry')
        mesh_c = SubElement(geometry_c, 'mesh')
        mesh_c.attrib = {'filename':'package://' + self.repo + self.name + '_m-binary.stl'}

        # print("\n".join(prettify(link).split("\n")[1:]))
        self.xml = "\n".join(prettify(link).split("\n")[1:])

In [5]:
def link_gen(dct, repo, link_dict, file_name):
    """This generates link urdf. Link_dict will be used in joint_gen.
    """
    with open(file_name, mode='w') as f:
        # about base_link
        link = Link(name='base_link', xyz=[0,0,0], repo=repo)
        link.gen_link_xml()
        f.write(link.xml)
        link_dict[link.name] = link.xyz
        # others
        for joint in dct:
            link = Link(name=dct[joint]['child'], xyz=dct[joint]['xyz'], repo=repo)
            link.gen_link_xml()
            f.write(link.xml)
            link_dict[link.name] = link.xyz
            

link_gen(joints_dict, repo, link_dict, file_name)

In [6]:
link_dict

{'back_leg_left': [0.068, -0.0344, -0.0338],
 'back_leg_right': [0.068, -0.0344, -0.0338],
 'back_wheel_left': [-0.126994, -0.130414, 0.085529],
 'back_wheel_right': [0.131994, -0.130414, 0.085529],
 'base_link': [0, 0, 0],
 'front_leg_left': [-0.099999, 0.127661, 0.022845],
 'front_leg_right': [0.099999, 0.127661, 0.022845],
 'front_wheel_left': [-0.136399, 0.212358, 0.095775],
 'front_wheel_right': [0.136999, 0.212358, 0.095775],
 'middle_wheel_left': [-0.135999, 0.040876, 0.093783],
 'middle_wheel_right': [0.135999, 0.040876, 0.093783]}

In [7]:
p = {'a':[1,3,4]}
p.keys()[0]

'a'

In [8]:
class Joint:
    def __init__(self, name, xyz, axis, parent, child, type_='continuous'):
        self.name = name
        self.type = type_
        self.xyz = xyz
        self.parent = parent
        self.child = child
        self.axis = axis
        self.xml = None
    
    def gen_joint_xml(self):
        """generate the xml strings of joints
        """
        joint = Element('joint')
        joint.attrib = {'name':self.name}
        
        origin = SubElement(joint, 'origin')
        origin.attrib = {'xyz':' '.join([str(_) for _ in self.xyz]), 'rpy':'0 0 0'}
        parent = SubElement(joint, 'parent')
        parent.attrib = {'link':self.parent}
        child = SubElement(joint, 'child')
        child.attrib = {'link':self.child}
        axis = SubElement(joint, 'axis')
        axis.attrib = {'xyz':' '.join([str(_) for _ in self.axis])}
        
        # print("\n".join(prettify(joint).split("\n")[1:]))
        self.xml = "\n".join(prettify(joint).split("\n")[1:])

In [9]:
def joint_gen(dct, repo, link_dict, file_name):
    with open(file_name, mode='a') as f:
        for j in dct:
            parent = dct[j]['parent']
            child = dct[j]['child']
            xyz = [round(p-c, 6) for p, c in \
                   zip(link_dict[parent], link_dict[child])]  # xyz = paret - child
            joint = Joint(name=j, xyz=xyz, axis=dct[j]['axis'],\
                          parent=parent, child=child)
            joint.gen_joint_xml()
            f.write(joint.xml)
        
joint_gen(joints_dict, repo, link_dict, file_name)

In [10]:
link_dict

{'back_leg_left': [0.068, -0.0344, -0.0338],
 'back_leg_right': [0.068, -0.0344, -0.0338],
 'back_wheel_left': [-0.126994, -0.130414, 0.085529],
 'back_wheel_right': [0.131994, -0.130414, 0.085529],
 'base_link': [0, 0, 0],
 'front_leg_left': [-0.099999, 0.127661, 0.022845],
 'front_leg_right': [0.099999, 0.127661, 0.022845],
 'front_wheel_left': [-0.136399, 0.212358, 0.095775],
 'front_wheel_right': [0.136999, 0.212358, 0.095775],
 'middle_wheel_left': [-0.135999, 0.040876, 0.093783],
 'middle_wheel_right': [0.135999, 0.040876, 0.093783]}

In [11]:
pa = [1, 2, 3]
ci = [4, 5, 6]
[p-c for p, c in zip(pa, ci)]

[-3, -3, -3]